#shapes dataset

In [1]:
!wget http://www.kasprowski.pl/datasets/shapes.zip

--2024-01-28 17:23:00--  http://www.kasprowski.pl/datasets/shapes.zip
Resolving www.kasprowski.pl (www.kasprowski.pl)... 94.152.164.213
Connecting to www.kasprowski.pl (www.kasprowski.pl)|94.152.164.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2827519 (2.7M) [application/zip]
Saving to: ‘shapes.zip’

shapes.zip          100%[===================>]   2.70M  8.21MB/s    in 0.3s    

2024-01-28 17:23:00 (8.21 MB/s) - ‘shapes.zip’ saved [2827519/2827519]



In [2]:
!unzip 'shapes.zip'

Archive:  shapes.zip
   creating: shapes/circles/
  inflating: shapes/circles/image100.jpg  
  inflating: shapes/circles/image101.jpg  
  inflating: shapes/circles/image102.jpg  
  inflating: shapes/circles/image103.jpg  
  inflating: shapes/circles/image104.jpg  
  inflating: shapes/circles/image105.jpg  
  inflating: shapes/circles/image106.jpg  
  inflating: shapes/circles/image107.jpg  
  inflating: shapes/circles/image108.jpg  
  inflating: shapes/circles/image109.jpg  
  inflating: shapes/circles/image110.jpg  
  inflating: shapes/circles/image111.jpg  
  inflating: shapes/circles/image112.jpg  
  inflating: shapes/circles/image113.jpg  
  inflating: shapes/circles/image114.jpg  
  inflating: shapes/circles/image115.jpg  
  inflating: shapes/circles/image116.jpg  
  inflating: shapes/circles/image117.jpg  
  inflating: shapes/circles/image118.jpg  
  inflating: shapes/circles/image119.jpg  
  inflating: shapes/circles/image120.jpg  
  inflating: shapes/circles/image121.jpg  
  in

In [3]:
!pip install tensorflow

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

# Define the directory where the data is located
dataset_dir = '/content/shapes'

# Initialize the image data generator with rescaling
data_gen = ImageDataGenerator(rescale=1./255)

# Prepare the generator for the dataset
data_generator = data_gen.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary',
    seed=42)

# Load all the data in memory (since it's a small dataset)
images, labels = next(data_generator)

# Convert labels to categorical (one-hot encoding)
labels = to_categorical(labels, num_classes=2)

# Split the data into training and test sets with stratification
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)

print('Training set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)


Found 200 images belonging to 2 classes.
Training set: (25, 64, 64, 3) (25, 2)
Test set: (7, 64, 64, 3) (7, 2)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# CNN Model
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')  # 2 classes
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [5]:
# Train the CNN model
cnn_history = cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.7389 - accuracy: 0.4800 - val_loss: 0.7303 - val_accuracy: 0.5714
Epoch 2/10
1/1 [==============================] - 0s 352ms/step - loss: 0.6992 - accuracy: 0.5600 - val_loss: 0.5996 - val_accuracy: 0.5714
Epoch 3/10
1/1 [==============================] - 0s 391ms/step - loss: 0.4810 - accuracy: 0.5600 - val_loss: 0.5256 - val_accuracy: 0.5714
Epoch 4/10
1/1 [==============================] - 0s 244ms/step - loss: 0.3621 - accuracy: 0.8800 - val_loss: 0.5321 - val_accuracy: 0.7143
Epoch 5/10
1/1 [==============================] - 0s 171ms/step - loss: 0.3238 - accuracy: 1.0000 - val_loss: 0.5244 - val_accuracy: 0.7143
Epoch 6/10
1/1 [==============================] - 0s 213ms/step - loss: 0.2684 - accuracy: 1.0000 - val_loss: 0.4784 - val_accuracy: 0.7143
Epoch 7/10
1/1 [==============================] - 0s 165ms/step - loss: 0.1898 - accuracy: 1.0000 - val_loss: 0.4499 - val_accuracy: 0.7143
Epoch 8/10
1/1 [=======

In [6]:
# Evaluate the model
cnn_scores = cnn_model.evaluate(X_test, y_test)

print('Test loss:', cnn_scores[0])
print('Test accuracy:', cnn_scores[1])


1/1 [==============================] - 0s 59ms/step - loss: 0.5283 - accuracy: 0.7143
Test loss: 0.5283328890800476
Test accuracy: 0.7142857313156128


In [7]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score

# Make predictions
predictions = cnn_model.predict(X_test)
# Convert predictions classes from one hot vectors to labels
y_pred = np.argmax(predictions, axis=1)
y_true = np.argmax(y_test, axis=1)

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Cohen's kappa
kappa_score = cohen_kappa_score(y_true, y_pred)

print('Confusion Matrix:\n', conf_matrix)
print('Cohen\'s Kappa:', kappa_score)


1/1 [==============================] - 0s 474ms/step
Confusion Matrix:
 [[4 0]
 [2 1]]
Cohen's Kappa: 0.36363636363636376
